# Importing Required lybraries

pandas: For handling CSV and Excel files.

yfinance: To fetch stock data.

openpyxl: For Excel file creation.

In [5]:
import yfinance as yf
import pandas as pd
from openpyxl import Workbook

# Defining funtions

In [6]:
def load_stock_data(file_path):
    """
    Load stock data from a CSV file.
    :param file_path: Path to the CSV file
    :return: DataFrame with Ticker and Weightage columns
    """
    try:
        data = pd.read_csv(file_path)
        return data[['Ticker', 'Weightage']].dropna()  # Remove unwanted columns or NaN rows
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        exit()

def get_user_inputs():
    """
    Accept user inputs for date range and investment amount.
    :return: start_date, end_date, investment_amount
    """
    try:
        start_date = input("Enter the start date (YYYY-MM-DD): ")
        end_date = input("Enter the end date (YYYY-MM-DD): ")
        investment_amount = float(input("Enter the total investment amount: "))
        if investment_amount <= 0:
            raise ValueError("Investment amount must be positive.")
        return start_date, end_date, investment_amount
    except ValueError as e:
        print(f"Invalid input: {e}")
        exit()

def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch historical closing prices for a given stock ticker.
    :param ticker: Stock ticker symbol
    :param start_date: Start date for the data
    :param end_date: End date for the data
    :return: Series of closing prices
    """
    try:
        ticker_with_suffix = f"{ticker}.NS"  # Add `.NS` suffix for NSE stocks
        stock = yf.Ticker(ticker_with_suffix)
        data = stock.history(start=start_date, end=end_date)
        return data['Close']  # Return only the closing prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.Series()

def format_results(stocks_data, closing_prices, investment_amount):
    """
    Format results into the required structure, including start and end dates.
    :param stocks_data: DataFrame with stock tickers and weightages
    :param closing_prices: Dictionary of closing prices for each ticker
    :param investment_amount: Total investment amount
    :return: Formatted DataFrame
    """
    final_results = []

    for _, row in stocks_data.iterrows():
        ticker = row['Ticker']
        weightage = row['Weightage']

        if ticker not in closing_prices or closing_prices[ticker].empty:
            print(f"Skipping {ticker} due to missing data.")
            continue

        # Calculate investment per stock
        allocated_amount = investment_amount * weightage
        
        # Fetch shares for start and end dates
        try:
            start_price = closing_prices[ticker].iloc[0]  # First date (start)
            end_price = closing_prices[ticker].iloc[-1]  # Last date (end)

            start_shares = allocated_amount / start_price
            end_shares = allocated_amount / end_price
        except Exception as e:
            print(f"Error calculating shares for {ticker}: {e}")
            continue

        # Add data to results
        final_results.append([ticker, weightage, start_shares, end_shares])

    # Create the DataFrame with appropriate columns
    formatted_df = pd.DataFrame(
        final_results,
        columns=['Ticker', 'Weightage', closing_prices[ticker].index[0].strftime('%Y-%m-%d'), 
                 closing_prices[ticker].index[-1].strftime('%Y-%m-%d')]
    )
    return formatted_df

def save_to_excel(results, output_file):
    """
    Save the calculated results to an Excel file.
    :param results: DataFrame with calculated shares
    :param output_file: Path to the output Excel file
    """
    try:
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            results.to_excel(writer, sheet_name='Investment Timetable', index=False)
        print(f"Results successfully written to {output_file}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")

def main():
    """
    Main function to execute the entire process.
    """
    file_path = 'Stocks.csv'  # Path to your CSV file
    output_file = 'output.xlsx'

    # 1) Load stock data
    stocks_data = load_stock_data(file_path)

    # 2) Get user inputs
    start_date, end_date, investment_amount = get_user_inputs()

    # 3) Fetch historical stock data
    closing_prices = {}
    for _, row in stocks_data.iterrows():
        ticker = row['Ticker']
        print(f"Fetching data for {ticker}...")
        closing_prices[ticker] = fetch_stock_data(ticker, start_date, end_date)

    # 4) Format results into the required structure
    formatted_results = format_results(stocks_data, closing_prices, investment_amount)

    #  5) Save results to Excel
    save_to_excel(formatted_results, output_file)

    #6) Display a preview of the results
    print("Preview of Results:")
    print(formatted_results.head())

# Entry point of the script

In [7]:
if __name__ == "__main__":
    main()

Enter the start date (YYYY-MM-DD):  2024-12-02
Enter the end date (YYYY-MM-DD):  2024-12-05
Enter the total investment amount:  100000


Fetching data for ADANIPORTS...
Fetching data for APOLLOHOSP...
Fetching data for ASIANPAINT...
Fetching data for AXISBANK...
Fetching data for BAJAJFINSV...
Fetching data for BAJFINANCE...
Fetching data for BHARTIARTL...
Fetching data for BPCL...
Fetching data for BRITANNIA...
Fetching data for CIPLA...
Fetching data for COALINDIA...
Fetching data for DIVISLAB...
Fetching data for DRREDDY...
Fetching data for EICHERMOT...
Fetching data for GRASIM...
Fetching data for HCLTECH...
Fetching data for HDFCBANK...
Fetching data for HDFCLIFE...
Fetching data for HEROMOTOCO...
Fetching data for HINDALCO...
Fetching data for HINDUNILVR...
Fetching data for ICICIBANK...
Fetching data for INDUSINDBK...
Fetching data for INFY...
Fetching data for ITC...
Fetching data for JSWSTEEL...
Fetching data for KOTAKBANK...
Fetching data for LT...
Fetching data for MARUTI...
Fetching data for NESTLEIND...
Fetching data for NTPC...
Fetching data for ONGC...
Fetching data for POWERGRID...
Fetching data for REL